In [1]:
import pickle
# Load data (deserialize)
with open('mens_alphas_sep_2020.pickle', 'rb') as handle:
    player_alphas = pickle.load(handle)

In [2]:
import pandas as pd
import numpy as np

In [3]:
mens_df = pd.read_csv('../data/mens.csv',header=0,parse_dates=["Date"])
womens_df = pd.read_csv('../data/womens.csv',header=0,parse_dates=["Date"])

# Remove walkovers
mens_df = mens_df[mens_df['Comment']!='Walkover']
womens_df = womens_df[womens_df['Comment']!='Walkover']

K:\Programs\Python\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (11,12,15,16,17,18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
K:\Programs\Python\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (25,28,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
mens_df = mens_df[mens_df['Date'] > pd.to_datetime('09/01/2020')]

In [5]:
def lookup_alphas(row,player_alphas):
    try:
        w_alpha = player_alphas[row['Date']][row['Winner']]
        l_alpha = player_alphas[row['Date']][row['Loser']]
    except:
        w_alpha = np.nan
        l_alpha = np.nan
    return w_alpha,l_alpha

In [6]:
mens_df[['W_alpha','L_alpha']] = mens_df.apply(lambda x:lookup_alphas(x,player_alphas),axis=1,result_type="expand")

In [7]:
mens_df = mens_df[['Winner','Loser','WRank','LRank','W_alpha','L_alpha']]
mens_df.dropna(inplace=True)

In [8]:
mens_df['Correct'] = mens_df['W_alpha'] > mens_df['L_alpha']
mens_df['p'] = mens_df[['W_alpha','L_alpha']].max(axis=1)/(mens_df['W_alpha']+mens_df['L_alpha'])
mens_df['d_alpha'] = abs(mens_df['W_alpha']-mens_df['L_alpha'])
mens_df

,Winner,Loser,WRank,LRank,W_alpha,L_alpha,Correct,p,d_alpha
52808,Hurkacz H.,Sousa J.,33,68,1.239643,0.982741,True,0.557799,0.256902
52809,Ofner S.,Albot R.,159,70,0.825863,1.056666,False,0.561301,0.230803
52810,Kecmanovic M.,Nishikori K.,47,34,0.874710,1.238808,False,0.586136,0.364098
52811,Pella G.,Nishioka Y.,36,49,1.410557,1.175592,True,0.545428,0.234965
52812,Hanfmann Y.,Bublik A.,118,54,0.755405,0.810345,False,0.517544,0.054940
...,...,...,...,...,...,...,...,...,...
53463,Djokovic N.,Zverev A.,1,7,2.052166,1.459409,True,0.584400,0.592757
53464,Medvedev D.,Schwartzman D.,4,9,1.486058,2.063613,False,0.581353,0.577555
53465,Thiem D.,Djokovic N.,3,1,1.683302,1.651043,True,0.504837,0.032258
53466,Medvedev D.,Nadal R.,4,2,1.128759,2.206208,False,0.661538,1.077449


In [14]:
for p0 in [0.5,0.6,0.65,0.7,0.8]:
    c_df = mens_df[mens_df['p']>p0]
    accuracy = c_df['Correct'].sum()/c_df.shape[0]
    print(accuracy,c_df.shape[0],p0)

0.5647058823529412 510 0.5
0.5882352941176471 119 0.6
0.6 45 0.65
0.6111111111111112 18 0.7
0.0 2 0.8
